In [1]:
# train simple rl again

In [2]:
import gym
import numpy as np
import argparse
from itertools import count
from collections import deque

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from drawnow import drawnow
import matplotlib.pyplot as plt

In [5]:
class FLAGS():
    lr = 1e-3 # learning rate
    gamma = 0.9 # discount factor
    epsilon_start = 0.9 # epsilon start
    epsilon_end = 0.05 # epsilon end
    target_update = 10 # target update 
    batch_size = 512 # batch size
    max_episode = 512 # number of episodes to train

In [6]:
cfg = FLAGS()

In [27]:
env = gym.make('CartPole-v0')
device = torch.device("cuda:02")

In [28]:
class Memory(object):
    def __init__(self, memory_size=10000):
        self.memory = deque(maxlen=memory_size)
        self.memory_size = memory_size
        
    def __len__(self):
        return len(self.memory)
    
    def append(self, item):
        self.memory.append(item)
        
    def sample_batch(self, batch_size):
        idx = np.random.permutation(len(self.memory))[:batch_size]
        return [self.memory[i] for i in idx]

In [29]:
# to memorize the past transitions
memory = Memory(10000)

In [30]:
class DQN(nn.Module):
    def __init__(self):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(4, 64)
        self.fc3 = nn.Linear(64, 2)
    def forward(self, x):
        x = F.elu(self.fc1(x))
        x = self.fc3(x)
        return xfactionzzx

In [35]:
policy_net = DQN().to(device)
target_net = DQN().to(device)
target_net.load_state_dict(policy_net.state_dict())

<All keys matched successfully>

In [85]:
def update_network(states, actions, next_states, rewards, dones):
    
    # update network:
    
    state_action_values = policy_net(states.to(device)).gather(1, actions[:, None].long().to(device)).squeeze()
    next_state_values = torch.max(target_net(next_states.to(device)), dim=1)[0].detach()
    expected_state_action_values = rewards.to(device) + next_state_values * (1 - dones.to(device)) * cfg.gamma

    expected_state_action_values = expected_state_action_values.to(device)

    loss = F.mse_loss(state_action_values, expected_state_action_values)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [113]:
next_states.shape

torch.Size([512, 4])

In [119]:
a = torch.max(target_net(torch.rand(2,4).to(device)), dim=1)

In [120]:
a

torch.return_types.max(
values=tensor([ 6.2942, -4.2964], device='cuda:2', grad_fn=<MaxBackward0>),
indices=tensor([1, 1], device='cuda:2'))

In [121]:
a[0]

tensor([ 6.2942, -4.2964], device='cuda:2', grad_fn=<MaxBackward0>)

In [111]:
policy_net(torch.rand(2,4).to(device)).gather(1, torch.tensor([1,1])[:, None].long().to(device)).squeeze()

tensor([5.1487, 7.6316], device='cuda:2', grad_fn=<SqueezeBackward0>)

In [86]:
policy_net.forward(torch.rand((1,4)).to(device))

tensor([[2.5736, 3.3400]], device='cuda:2', grad_fn=<AddmmBackward>)

In [87]:
def get_action(state, epsilon):
    with torch.no_grad():
        greedy_action = torch.argmax(policy_net(state.to(device)), dim=1).item()
        random_action = np.random.randint(0, 2)
    return random_action if np.random.rand() < epsilon else greedy_action

In [88]:
optimizer = optim.RMSprop(policy_net.parameters(), lr=cfg.lr, weight_decay=1e-4)

In [89]:
last_score_plot = [0]
avg_score_plot = [0]

In [93]:
cfg.epsilon_end, cfg.epsilon_start

(0.05, 0.9)

In [100]:
z = 512
a = (cfg.epsilon_end-cfg.epsilon_start)*(z/512)+cfg.epsilon_start
a

0.050000000000000044

In [103]:
torch.tensor(state).float()[None, :]

tensor([[-0.0333, -0.5662,  0.1039,  0.9675]])

In [97]:
for i in range(cfg.max_episode):
    episode_durations = 0
    state = env.reset()
    epsilon = (cfg.epsilon_end - cfg.epsilon_start) * (i / cfg.max_episode) + cfg.epsilon_start

    for t in count():
        action = get_action(torch.tensor(state).float()[None, :], epsilon)
        next_state, reward, done, _ = env.step(action)

        memory.append([state, action, next_state, reward, done])
        state = next_state

        if len(memory) > cfg.batch_size:
            states, actions, next_states, rewards, dones = \
            map(lambda x: torch.tensor(x).float(), zip(*memory.sample_batch(cfg.batch_size)))

            update_network(states, actions, next_states, rewards, dones)

        if done:
            episode_durations = t + 1
            avg_score_plot.append(avg_score_plot[-1] * 0.99 + episode_durations * 0.01)
            last_score_plot.append(episode_durations)
            #drawnow(draw_fig)
            break

    # Update the target network
    if i % cfg.target_update == 0:
        target_net.load_state_dict(policy_net.state_dict())
        
        
    if i % 10 == 0:
        print(i)

torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([5

torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([512, 4])
torch.Size([5

KeyboardInterrupt: 